In [7]:
import pandas as pd
import geojson
import openrouteservice
import time
import os
import folium
import json
from shapely.geometry import Point, Polygon
from pyproj import Geod
from math import radians, cos, sin, asin, sqrt
from vincenty import vincenty


import os
from dotenv import load_dotenv
load_dotenv('.env')

True

In [8]:
# authentication OpenRouteService
secret = os.getenv("ORS_SECRET")
ors = openrouteservice.Client(key=secret)

In [9]:
# load CSV
poi = pd.read_csv('./dummy/poi.csv')

posters = pd.read_csv('./dummy/p.csv')


posters['ID'] = posters['ID'].apply(lambda x : 'P' + str(x))

posters = posters.set_index('ID')

poi

,poi_name,latitude,longitude
0,Antwerp Kinepolis,51.245236,4.415847


In [10]:
# extract coords
poi_coords = poi[["poi_name", "longitude", 'latitude']].values.tolist()
poster_coords= posters.T.to_dict('list')

poster_coords

{'P22': [51.278824, 4.409519],
 'P49': [51.182657, 4.3729141],
 'P65': [51.185132, 4.3442028],
 'P77': [51.171207, 4.364278],
 'P78': [51.171212, 4.3644414],
 'P80': [51.16932, 4.362545638],
 'P84': [51.183229, 4.372675],
 'P122': [51.238631, 4.453698],
 'P132': [51.251168, 4.449170522],
 'P137': [51.253444, 4.435109],
 'P141': [51.253825, 4.4355804],
 'P159': [51.259815, 4.4495068],
 'P189': [51.164014, 4.391996958],
 'P219': [51.163794, 4.4004772],
 'P220': [51.18165, 4.391218],
 'P239': [51.157605, 4.371189],
 'P243': [51.170694, 4.383201147],
 'P245': [51.168411, 4.3902351],
 'P249': [51.240375, 4.4369445],
 'P87': [51.161472, 4.3534352],
 'P135': [51.260159, 4.4457976],
 'P143': [51.255159, 4.4404288],
 'P157': [51.256377, 4.4342995],
 'P162': [51.177503, 4.3988934],
 'P204': [51.172804, 4.4033597],
 'P232': [51.176284, 4.4252168],
 'P181': [51.159779, 4.3999749],
 'P45': [51.176821, 4.3711708],
 'P156': [51.244982, 4.441263],
 'P258': [51.199533, 4.4395575],
 'P261': [51.187075, 

In [11]:
# make geojson files for each pair of coords
# for item in poi_coords:
   
#     isochrone = ors.isochrones(locations=[item], range=[600])
#     with open('./dummy/test_geo.geojson', 'w') as f:
#         json.dump(isochrone, f, ensure_ascii=False,indent=4)

In [12]:
# load geojson file into the program and check that it is a valid object
with open('./dummy/test_geo.geojson', 'r') as f:
    gj = geojson.load(f)

gj.is_valid

True

In [13]:
# reverse coords for use in folium
for item in poi_coords:
    item.reverse()


In [14]:
map = folium.Map(location=(poi_coords[0][:2]), tiles="cartodbpositron", min_zoom=11)

folium.Marker(poi_coords[0][:2]).add_to(map)

# for item in poster_coords:
    

folium.features.GeoJson(gj).add_to(map)

In [15]:
drivetime = Polygon(gj['features'][0]['geometry']['coordinates'][0])


count = 0
for id, coords in poster_coords.items():
    long = coords[1]
    lat = coords[0]
    point = Point(long,lat)

    if point.within(drivetime) or point.touches(drivetime):
        folium.Marker(location= [lat, long],popup=id, icon= folium.Icon(color='red', icon='star') ).add_to(map)
        folium.Circle([lat, long],radius=300, color='red', fill_color = 'red').add_to(map)
        count += 1
    else:
        folium.Marker(location= [lat, long],popup=item[2], icon= folium.Icon(color='cadetblue', icon='star') ).add_to(map)
print(count)

map


52


In [16]:

# returns area in square-degrees
drivetime.area

0.006820629596499976

In [17]:
# https://stackoverflow.com/questions/23697374/calculate-polygon-area-in-planar-units-e-g-square-meters-in-shapely

geod = Geod(ellps="WGS84")

area, perimeter = geod.geometry_area_perimeter(drivetime)

sqkm = area / 1000000

sqkm

52.991663173533325

In [18]:
density = count / abs(sqkm)

density

0.9812864304657527

In [19]:
A = (posters.at['P485', 'latitude'], posters.at['P485', 'longitude'])
B = (posters.at['P889', 'latitude'], posters.at['P889', 'longitude'])
dist = vincenty(A,B)

dist

0.226892